In [24]:
import pandas as pd
import random
import time
import hashlib

from kafka import KafkaProducer
from confluent_kafka import Producer
import socket
import json
import os

from kafka_auth import conf

In [2]:
# Leer el archivo CSV
df = pd.read_csv('Sales_simulation.csv', usecols=['Category', 'Name', 'Purchase Price', 'Delivery Price'])


In [3]:
cities = ['Bogotá', 'Medellín', 'Cali', 'Bucaramanga', 'Barranquilla']
payment_online = ['Credit_card', 'PSE']
payment_store = ['Cash', 'Nequi', 'Daviplata', 'Credit_card']
source = ['Facebook', 'Instagram', 'Organic', 'Twitter', 'Influencer_1', 'Influencer_2', 'Influencer_3', 'Influencer_4']
status_purchase = ['COMPLETED', 'FAILED_CHECKOUT', 'FAILED_API_RESPONSE', 'INSUFICCIENT_FUNDS', 'COMPLETED', 'COMPLETED', 'COMPLETED', 'COMPLETED', 'COMPLETED', 'COMPLETED', 'FAILED_API_RESPONSE', 'INSUFICCIENT_FUNDS', 'USER_ERROR', 'FRAUD', 'COMPLETED', 'COMPLETED', 'COMPLETED']

In [4]:
bog_coords = [(4.651600960108946, -74.12628850377475), (4.661984763443271, -74.13466548843223), (4.647569693587232, -74.10186525959672)]
buc_coords = [(7.099917472863198, -73.10730617492302), (7.07243688331635, -73.10525936227518)]
cali_coords = [(3.4287786094866717, -76.53749228193497), (3.4148803158142966, -76.54041613631288), (3.4164091379805432, -76.54751692551635)]
bar_coords = [(11.014167139030025, -74.82747678131524), (11.004041676077495, -74.83545204769058), (10.990580146449894, -74.78876005521157)]
mede_coords = [(6.163315879042265, -75.6052691935286), (6.17784573272914, -75.59141059178306), (6.198053256721469, -75.5733524126965)]

In [5]:
# Función para obtener los créditos a favor del usuario
def get_user_credits():
    return random.choice([0, 2000, 5000, 10000, 15000, 20000])

In [6]:
def get_coords(city):
    
    if city == 'Bogotá' :
        coords = random.choice(bog_coords)
    elif city == 'Medellín' :
        coords = random.choice(mede_coords)
    elif city == 'Cali' :
        coords = random.choice(cali_coords)
    elif city == 'Bucaramanga' :
        coords = random.choice(buc_coords)
    elif city == 'Barranquilla' :
        coords = random.choice(bar_coords)
        
    return coords 

In [7]:
def get_pay_method(source,status_purchase,payment_online,payment_store):
    
    if source == 'Organic' :
        
        payment = random.choice(payment_store)
        status = 'COMPLETED'
        order_type = 'STORE'
    
    elif source != 'Organic' :
        
        payment = random.choice(payment_online)
        status = random.choice(status_purchase)
        order_type = 'ONLINE'
    
    return payment,status,order_type   

In [8]:
# Función para obtener la propina para el domiciliario
def get_tip():
    return random.choice([0, 2000, 3000, 5000])


In [45]:
df  


,Category,Name,Purchase Price,Delivery Price
0,Saludable · Desayunos,Distrito Gourmet,10700,3700
1,Postres,Toledo Pastelería,53900,6800
2,Postres · Helados,Mahé,7500,6000
3,Hamburguesas · Saludable,Restaurant Inn,23000,6100
4,Postres · Desayunos,Maria Luisa - Postres,9000,4300
...,...,...,...,...
23225,Desayunos · Típica,La Frijolada,8900,4500
23226,Típica,Sopitas y Más,31800,4200
23227,Desayunos · Típica,Calentao Express,23000,6000
23228,Desayunos · Árabe,Boniato Sabores Vivos,37900,5300


In [46]:
conf

{'bootstrap.servers': 'pkc-p11xm.us-east-1.aws.confluent.cloud:9092',
 'security.protocol': 'SASL_SSL',
 'sasl.mechanisms': 'PLAIN',
 'sasl.username': '7FERWKPQY5EE7ZG5',
 'sasl.password': 'm9Us1yrmUU3JkG32Nc2ovCBFdxEMeX5WH0xbFGM3+aX4WHDu2ciCE6X5xHrnXc1r'}

In [47]:
# conf = ccloud_lib.read_ccloud_config('librdkafka.config')

# # Create Producer instance
# producer_conf = ccloud_lib.pop_schema_registry_params_from_config(conf)
# producer = Producer(producer_conf)

producer = Producer(conf)

%4|1710191074.707|TERMINATE|rdkafka#producer-1| [thrd:app]: Producer terminating with 1 message (429 bytes) still in queue or transit: use flush() to wait for outstanding message delivery


In [51]:
delivered_records = 0
topic = 'rappi_orders' 


x = 1

data_purchase = []

while x > 0:
    # Muestra aleatoria de una fila del DataFrame df
    random_row = df.sample()

    date = pd.to_datetime('today').strftime("%Y-%m-%d %H:%M:%S")  # Obtiene la fecha y hora actual
    product = random_row['Name'].values[0]  # Obtiene el valor de la columna 'Name' de la fila aleatoria
    pricing = random_row['Purchase Price'].values[0]  # Obtiene el valor de la columna 'Purchase Price' de la fila aleatoria
    delivery_price = random_row['Delivery Price'].values[0]  # Obtiene el valor de la columna 'Delivery Price' de la fila aleatoria
    category = random_row['Category'].values[0]  # Obtiene el valor de la columna 'Category' de la fila aleatoria
    source_temp = random.choice(source)  # Elige aleatoriamente una fuente de compra de la lista de fuentes disponibles
    pay = get_pay_method(source_temp, status_purchase, payment_online, payment_store)  # Determina el método de pago, el estado de la compra y el tipo de orden
    city = random.choice(cities)  # Elige aleatoriamente una ciudad de la lista de ciudades disponibles

    # Obtiene los créditos a favor del usuario y la propina para el domiciliario
    user_credits = get_user_credits()
    tip = get_tip()

    # Calcula el tiempo de entrega estimado
    delivery_time = random.randint(15, 60)  # Estimación del tiempo de entrega en minutos

    # Crea un diccionario con la información de la compra
    purchase = {
        'purchase_ID': str(hashlib.sha256(f"{random_row.index.values[0]} {product} {pricing} {delivery_price} {date} {source_temp} {pay[1]}".encode('utf-8')).hexdigest())[:10],
        'Product_name': product,
        'Pricing': str(pricing),
        'Delivery_Price': str(delivery_price),
        'Category': category,
        'Payment_Method': pay[0],
        'Status': pay[1],
        'Order_Type': pay[2],
        'City': city,
        'Latitud': str(get_coords(city)[0]),
        'Longitud': str(get_coords(city)[1]),
        'Source': source_temp,
        'User_Credits': user_credits,
        'Tip': tip,
        'Estimated_Delivery_Time': delivery_time,  # Tiempo de entrega estimado en minutos
        'Created_at': date
    }

    #data_purchase.append(pd.DataFrame(purchase))

    record_key = "Purchase_simulator"
    record_value =  json.dumps(purchase).encode('utf-8')
    print("Producing record: {}\t{}".format(record_key, record_value))
    producer.produce(topic, key=record_key, value=record_value)
    producer.poll(0)
    delivered_records += 1

    time.sleep(random.choice([1, 1.5]))

    producer.flush()

Producing record: Purchase_simulator	b'{"purchase_ID": "51a166108e", "Product_name": "El Pan de Phillippe", "Pricing": "8900", "Delivery_Price": "4600", "Category": "Desayunos \\u00b7 Panader\\u00eda", "Payment_Method": "PSE", "Status": "COMPLETED", "Order_Type": "ONLINE", "City": "Bogot\\u00e1", "Latitud": "4.661984763443271", "Longitud": "-74.10186525959672", "Source": "Influencer_2", "User_Credits": 5000, "Tip": 2000, "Estimated_Delivery_Time": 36, "Created_at": "2024-03-11 18:08:27"}'
Producing record: Purchase_simulator	b'{"purchase_ID": "58c5692c21", "Product_name": "Maracuy\\u00e1", "Pricing": "13000", "Delivery_Price": "6700", "Category": "Saludable \\u00b7 Desayunos", "Payment_Method": "Credit_card", "Status": "COMPLETED", "Order_Type": "ONLINE", "City": "Medell\\u00edn", "Latitud": "6.163315879042265", "Longitud": "-75.59141059178306", "Source": "Influencer_2", "User_Credits": 2000, "Tip": 2000, "Estimated_Delivery_Time": 35, "Created_at": "2024-03-11 18:08:28"}'
Producing re

KeyboardInterrupt: 

%5|1710202534.787|REQTMOUT|rdkafka#producer-2| [thrd:sasl_ssl://b14-pkc-p11xm.us-east-1.aws.confluent.cloud:9092/14]: sasl_ssl://b14-pkc-p11xm.us-east-1.aws.confluent.cloud:9092/14: Timed out MetadataRequest in flight (after 60077ms, timeout #0)
%4|1710202534.787|REQTMOUT|rdkafka#producer-2| [thrd:sasl_ssl://b14-pkc-p11xm.us-east-1.aws.confluent.cloud:9092/14]: sasl_ssl://b14-pkc-p11xm.us-east-1.aws.confluent.cloud:9092/14: Timed out 1 in-flight, 0 retry-queued, 0 out-queue, 0 partially-sent requests
%3|1710202534.787|FAIL|rdkafka#producer-2| [thrd:sasl_ssl://b14-pkc-p11xm.us-east-1.aws.confluent.cloud:9092/14]: sasl_ssl://b14-pkc-p11xm.us-east-1.aws.confluent.cloud:9092/14: 1 request(s) timed out: disconnect (after 838309ms in state UP, 1 identical error(s) suppressed)
%5|1710202595.384|REQTMOUT|rdkafka#producer-2| [thrd:sasl_ssl://b5-pkc-p11xm.us-east-1.aws.confluent.cloud:9092/5]: sasl_ssl://b5-pkc-p11xm.us-east-1.aws.confluent.cloud:9092/5: Timed out MetadataRequest in flight (afte

In [41]:
print('\n')
print('\n')
print("{} messages were produced to topic {}!".format(delivered_records, topic))
print('\n')
print('\n')





4 messages were produced to topic topic_rappi!




